In [1]:
numbers = {}
numbers[(1,1)] = 20151125

In [10]:
def value(indices):
    if indices in numbers:
        return numbers[indices]
    r, c = indices
    if c==1:
        r_, c_ = 1, r-1
    else:
        r_, c_ = r+1, c-1
    previous = value((r_, c_))
    computed = (previous*252533)%33554393
    numbers[(r,c)] = computed
    return computed

In [21]:
for i in range(1,10000):
    r, c = i, 1
    while c <= i:
        value((r, c))
        r, c = r-1, c+1

In [24]:
value((2981,3075))

9132360